# Classification Model
* File Name: Reference_implementation of BFSI 
* Date of creation(dd-mm-yyyy) : 30-03-2023
* Author Name/Dept : AIShield
* Organization : BGSW
* Description : Source Code of reference implementation of AIShield API
* Copyright : Copyright 2022 Bosch Global Software Technologies Private Limited. All Rights Reserved.

### Metadata
* Dataset: Banking Marketing Campagin
* Size of dataset:41188, 18
* Number of class : 2
* Original Model: XGBOOST 

### Outcomes
* Accuracy of model: 0.92


In [1]:
"""
Description: commands to install aishield
"""

#!pip install aishield

'\nDescription: commands to install aishield\n'

In [2]:
"""
Description: commands to install all the packages, remove comments to install all the libraries
"""
# ! pip install xgboost==1.6.2
# ! pip install pandas==1.1.5
# ! pip install scikit-learn==1.0.2
# ! pip install numpy==1.22
# ! pip install pyminizip
# !pip install requests==2.28.0
# !pip install humanfriendly==9.2

'\nDescription: commands to install all the packages, remove comments to install all the libraries\n'

### 1.0 Import libraries

In [1]:
import os
import copy
import json
import time
import pickle
import shutil
import requests
import py_compile
import pyminizip
import zipfile
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score


#import aishield libraries
import aishield as ais

### 2.0 Data Loading and Preprocessing

#### Download dataset from the link https://archive.ics.uci.edu/dataset/222/bank+marketing 
1. then download the bank-additional.zip file and extract it , If running in colab, then uncomment the below cell to upload the downloaded zip file to colab

In [88]:
# # To upload the downloaded zip file in Colab, please uncomment the below cell.
# from google.colab import files
# import zipfile
# import io

# uploaded = files.upload()

# def extract_nested_zip(zip_file):
#     # Look for nested zip files
#     nested_zip_names = [name for name in zip_file.namelist() if name.endswith(".zip")]

#     if not nested_zip_names:
#         print("No nested zip files found.")
#         return

#     for nested_zip_name in nested_zip_names:
#         with zip_file.open(nested_zip_name) as nested_zip_file:
#             with io.BytesIO(nested_zip_file.read()) as nested_buffer:
#                 with zipfile.ZipFile(nested_buffer, 'r') as nested_zip:
#                     # Extract the contents of the nested zip file
#                     nested_zip.extractall()

# for name, content in uploaded.items():
#     with io.BytesIO(content) as buffer:
#         with zipfile.ZipFile(buffer, 'r') as outer_zip:
#             # Extract the contents of the outer zip file
#             outer_zip.extractall()

#             # Extract nested zip files recursively
#             extract_nested_zip(outer_zip)

# print("Extraction completed.")


In [27]:
'''
Description : Loading credit card fraud detection dataset
'''
df = pd.read_csv('bank-additional/bank-additional-full.csv',delimiter=";")

In [28]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [29]:
df.replace(['basic.6y','basic.4y', 'basic.9y'], 'basic', inplace=True)

In [30]:
columns = ['day_of_week','month']
df.drop(columns, axis=1, inplace=True)

In [31]:
'''Description : Label Encoding the Categorical Columns'''

le =LabelEncoder()
df.job = le.fit_transform(df.job)
df.marital = le.fit_transform(df.marital)
df.education = le.fit_transform(df.education)
df.housing = le.fit_transform(df.housing)
df.loan = le.fit_transform(df.loan)
df.poutcome = le.fit_transform(df.poutcome)
df.contact = le.fit_transform(df.contact)
df.default = le.fit_transform(df.default)
df.y = le.fit_transform(df.y)

In [32]:
df.head()

,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,3,1,0,0,0,0,1,261,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0
1,57,7,1,1,1,0,0,1,149,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0
2,37,7,1,1,0,2,0,1,226,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0
3,40,0,1,0,0,0,0,1,151,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0
4,56,7,1,1,0,0,2,1,307,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0


In [33]:
output = 'y'
 
X = df.loc[:, df.columns != output]
y = df['y']

In [34]:
'''
Description : Splitting data for validation
'''
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 1)

In [35]:
'''
Description : Check size of dataset
'''
print("shape of x_train: ", X_train.shape)
print("shape of y_train: ", y_train.shape)
print("shape of x_test: ", X_test.shape)
print("shape of y_test: ", y_test.shape)

shape of x_train:  (30891, 18)
shape of y_train:  (30891,)
shape of x_test:  (10297, 18)
shape of y_test:  (10297,)


In [36]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(X_train)
x_test = scaler.transform(X_test)

### 3.0 Model Development and Training

In [37]:
def make_directory(directory):
    """
    create directory

    Parameters
    ----------
    directorys : list containing the directorys path to create 
    Returns
    -------
    None.

    """
    for d in directory:
        if os.path.isdir(d):
            print("directory {} already exist".format(d))
        if os.path.isdir(d)==False:
            os.mkdir(path=d)
            print("directory {} created successfully".format(d))

In [38]:
def delete_directory(directorys):
    """
    delete directory 

    Parameters
    ----------
    directorys : list containing the directorys to deleate along with all the files

    Returns
    -------
    None.

    """
    if len(directorys)>=1:
        for d in directorys:
            if os.path.isdir(d):
                try:
                    if os.path.isfile(d):
                        os.remove(path=d)
                    else:
                        shutil.rmtree(path=d)
                        print("Removed: {}".format(d))
                except:
                    print("Failed to removed: {}".format(d))
                

In [39]:
def make_archive(base_name,root_dir,zip_format='zip'):
    """
    created zip for given folder

    Parameters
    ----------
    base_name : name of zip file
    root_dir : directory to archive/zip
    zip_format : zip or tar 
        DESCRIPTION. The default is 'zip'.

    Returns
    -------
    None.

    """
    shutil.make_archive(base_name=base_name, format=zip_format, root_dir=root_dir)
    

In [86]:
"Description : Create data, model and label folder"
data_path=os.path.join(os.getcwd(),"data")
model_path=os.path.join(os.getcwd(),"model")
minmax_path=os.path.join(os.getcwd(),"minmax")
zip_path=os.path.join(os.getcwd(),"zip")
pyc_model_path=os.path.join(os.getcwd(),"pycmodel")
report_path = os.path.join(os.getcwd(), "reports")
#deleting folder
delete_directory(directorys=[data_path,model_path,minmax_path,zip_path,pyc_model_path,report_path])

#creating folder
make_directory([data_path,model_path,minmax_path,zip_path,pyc_model_path,report_path])

'Description : Create data, model and label folder'

In [41]:
def model():
    XG = XGBClassifier()
    return XG

In [42]:
"""
Description : Create architecture of the model
"""
model=model()
print(model)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=None,
              gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=None,
              reg_alpha=None, reg_lambda=None, ...)


In [43]:
"""
Description: Trainig the model 
"""

model.fit(x_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [44]:
"""
Saving the model under model directory
"""
modelname = "xgboost_evasion.pkl"
pickle.dump(model, open(model_path+"/"+modelname, 'wb'))

In [45]:
"""
Description: Get prediction on validation data
"""
pred=model.predict(x_test)

In [46]:
print('Accuracy score of the model is {}'.format(accuracy_score(y_test, pred)))
print('F1 score of the model is {}'.format(f1_score(y_test, pred)))
print('Confusion Matrix \n {}'.format(confusion_matrix(y_test, pred)))
print('Classification Report \n {}'.format(classification_report(y_test, pred)))

Accuracy score of the model is 0.9159949499854326
F1 score of the model is 0.601932811780948
Confusion Matrix 
 [[8778  354]
 [ 511  654]]
Classification Report 
               precision    recall  f1-score   support

           0       0.94      0.96      0.95      9132
           1       0.65      0.56      0.60      1165

    accuracy                           0.92     10297
   macro avg       0.80      0.76      0.78     10297
weighted avg       0.91      0.92      0.91     10297



### 4.0 Prepare Data , Model and MinMax

In [47]:
"""
Description: Zip data
"""
x = np.concatenate((x_train, x_test))
y = np.concatenate((y_train, y_test))
number_of_sample = 100


final_df = pd.DataFrame(list(x),columns=df.columns[:-1])
final_df[output] = y
final_df[:number_of_sample].to_csv(os.path.join(data_path,"banking_evasion.csv"),index=False)
make_archive(base_name=os.path.join(zip_path,"data"),root_dir=data_path,zip_format='zip')

In [48]:
"""Prepare MinMax"""

min_values = final_df.min().to_numpy()
max_values = final_df.max().to_numpy()
x= np.array([min_values,max_values])
df_m = pd.DataFrame(x,columns=final_df.columns)
df_m.to_csv(minmax_path+"/minmax.csv",index=False)

In [49]:
"""
Description: Zip Minmax
"""
make_archive(base_name=os.path.join(zip_path,"minmax"),root_dir=minmax_path,zip_format='zip')

In [50]:
"""
Description: Create encrypted Model
"""
def create_model_pyc(model, model_framework,pyc_model_path):
    
    if model_framework.lower() == "scikit-learn":

        python_code ='''
#importing libraries
import pickle
import zipfile
import os

"""
    class for base model
"""
#define class
class BaseModel():
    def __init__(self,model_path=""):

        """
        constructor for class

        Parameters
        ----------
        input_shape : TYPE, optional
            DESCRIPTION. The default is (100,1).
        Returns
        -------
        None.

        """
        self.model_path = model_path
        
    def load_protected_pickleModel(self,filename,password,picklemodelname):
      

        """
        model architecture

        Parameters
        ----------
        filename : string
            DESCRIPTION.zipped protected filename
            
        password : String
            DESCRIPTION.zipped file password
        
        picklemodelname  :String
            DESCRIPTION.pickle file name present in a ziiped protected
            
        Returns
        -------
        model : model
            DESCRIPTION.

        """
        filepath = os.path.join(self.model_path,filename)
        #print(filepath)
        #print(filename)
        with zipfile.ZipFile(filepath, 'r') as file:
            with file.open(picklemodelname,'r',pwd = bytes(password, 'utf-8')) as f:
                pck = pickle.load(f)
        return pck
    
    def predict(self,X):

        """
        predict for given data

        Parameters
        ----------
        X : numpy array 
            DESCRIPTION.

        Returns
        -------
        pred : numpy array
            DESCRIPTION.

        """
        filename = "modelencrypt.zip"
        password = "987654321"
        picklemodelname = "xgboost_evasion.pkl"
        model = self.load_protected_pickleModel(filename=filename,password=password,picklemodelname=picklemodelname)
        pred = model.predict(X)
        return pred'''
        # Writing to file
        with open("base_model.py", "w") as file:
            # Writing data to a file
            file.writelines(python_code)
         
        """
        Description: function to create .pyc file
        """
        py_compile.compile(file="base_model.py",cfile=pyc_model_path+'/base_model.pyc')
        
        """
        Description: function to create zipped password protected pickle model file
        """

        def zip_model(input_path,output_path,password,com_lvl=5):
            pyminizip.compress(input_path, None, output_path,
                               password, com_lvl)
            
        zip_model(input_path =os.path.join(model_path,modelname),output_path = pyc_model_path+"/modelencrypt.zip" ,password = "987654321",com_lvl=5)
            

In [51]:
"""
Description: Creating encrypted Model
"""

input_dimension = (number_of_sample,final_df.shape[1]-1)
number_of_classes = 2
model_framework = "scikit-learn"
model_encryption = 0 #0 for non encrypted file and 1 for encrypted (pyc) 
create_model_pyc(model,model_framework, pyc_model_path)

In [52]:
"""
Description: Zip model
"""
 # model_encryption 0 if model is uploaded directly as a zip, model_encryption 1 if model is encryted as .pyc and uploaded as a zip
if os.path.isfile(os.path.join(zip_path,"model.zip")):
    delete_directory(directorys=[os.path.join(zip_path,"model.zip")])
if model_encryption:
    make_archive(base_name=os.path.join(zip_path,"bfsi_pyc_model"),root_dir=pyc_model_path,zip_format='zip')
else:
    make_archive(base_name=os.path.join(zip_path,"model"),root_dir=model_path,zip_format='zip')

### 5.0 AIShield API Call


In [53]:
"""
Description: AIShield API call
"""

baseurl="xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx" # fill in API endpoint url from AIShield developer portal under API tab 
url=baseurl+"/api/ais/v1.5"
api_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxx"
org_id = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
 

#### 5.1 Perform Vulnerability Analysis through VulConfig Function


In [54]:
"""
Description: Files path
"""
data_path=os.path.join(zip_path,'data.zip')  # full path of data zip
label_path=os.path.join(zip_path,'minmax.zip') # full path of label zip
# model_path=os.path.join(zip_path,'bfsi_pyc_model.zip') # uncomment if model_encryption = 1 (pyc)
model_path=os.path.join(zip_path,'model.zip') # uncomment if model_encryption = 0 (unencrypted)

In [55]:
"""
Description: Initialize the AIShield API
"""
client = ais.AIShieldApi(api_url=url, api_key=api_key, org_id=org_id)

In [61]:
"""
Description: Define the task and analysis type
"""
task_type = ais.get_type("task", "tabular_classification")
analysis_type = ais.get_type("analysis", "evasion")

In [66]:
"""
Description: Perform model registration and upload the input artifacts
"""
status, job_details = client.register_model(task_type=task_type, analysis_type=analysis_type)
model_id = job_details.model_id
# print('Model id: {} \nInput artifacts will be uploaded as:\n data_upload_uri: {}\n label_upload_uri: {}'
#       '\n model_upload_uri: {}'.format(model_id, job_details.data_upload_uri, job_details.label_upload_uri,
#                                        job_details.model_upload_uri))

upload_status = client.upload_input_artifacts(job_details=job_details,
                                              data_path=data_path,
                                              minmax_path=label_path,
                                              model_path=model_path, )
print('Upload status: {}'.format(', '.join(upload_status)))

Upload status: data file upload successful, minmax file upload successful, model file upload successful


#### 5.2 Model Analysis


In [67]:
"""
Description: Specify the appropriate configs required for vulnerability analysis
"""

vuln_config = ais.VulnConfig(task_type=task_type,
                             analysis_type=analysis_type,
                             defense_generate=True)

vuln_config.input_dimensions = input_dimension # input dimension of banking marketing dataset
vuln_config.number_of_classes = number_of_classes  # number of classes of banking marketing dataset
vuln_config.is_category_columns = "yes"  # if cetgorical columns are present
vuln_config.categorical_columns_info = ["job","marital","education","default","housing","loan","contact","poutcome"]  # name of categorical columns
vuln_config.encryption_strategy = model_encryption  # value 0 (or) 1, if model is unencrypted or encrypted(pyc) respectively


print('TC-Extraction parameters are: \n {} '.format(vuln_config.get_all_params()))

TC-Extraction parameters are: 
 {'categorical_columns_info': 'job,marital,education,default,housing,loan,contact,poutcome', 'defense_bestonly': 'no', 'encryption_strategy': 0, 'input_dimensions': '(100, 18)', 'is_category_columns': 'yes', 'model_api_details': '', 'model_framework': 'scikit-learn', 'normalize_data': 'yes', 'number_of_classes': 2, 'use_model_api': 'no'} 


In [85]:
"""
Description: Run vulnerability analysis
"""
my_status, job_details = client.vuln_analysis(model_id=model_id, vuln_config=vuln_config)
my_job_id = job_details.job_id
print('status: {} \nJob_id: {} '.format(my_status, my_job_id))


status: success 
Job_id: xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx 


In [83]:
"""
Description: Monitor progress for given Job ID using the Link below
"""
print('Click on the URL to view Vulnerability Dashboard (GUI): {}'.format(job_details.job_monitor_uri))

'\nDescription: Monitor progress for given Job ID using the Link below\n'

In [71]:
"""
Description: Fetch Job status using Job ID
"""
my_status = client.job_status (job_id = my_job_id)
print('job status ', my_status)

2023-03-30 13:58:20,608 - INFO - Fetching job details for job id gAAAAABkJUgc3u_2lJiDYowUHLf4u_ciMcvaUghUJv1gON4R0c-xXid60uxSta8DjE4Xjnm9Y_Q1zteKY4MO-Nr8QsjuLwqUCA==
2023-03-30 13:58:26,179 - INFO - ModelExploration_Status:inprogress


2023-03-30 13:58:37,766 - INFO - ModelExploration_Status:completed
2023-03-30 13:58:37,766 - INFO - SanityCheck_Status:passed


2023-03-30 14:02:22,060 - INFO - EvasionAnalysis_Status:inprogress


2023-03-30 14:11:02,951 - INFO - EvasionAnalysis_Status:completed
2023-03-30 14:11:02,953 - INFO - DefenseReport_Status:inprogress


2023-03-30 14:12:43,919 - INFO - DefenseReport_Status:completed
2023-03-30 14:12:43,920 - INFO - Analysis completed for job id gAAAAABkJUgc3u_2lJiDYowUHLf4u_ciMcvaUghUJv1gON4R0c-xXid60uxSta8DjE4Xjnm9Y_Q1zteKY4MO-Nr8QsjuLwqUCA==


job run completed
job status  success


### Saving the Artifacts and the Reports


In [73]:
"""
Description: Creating a directory to save the defense artifacts
"""
OUTPUT_PATH = os.path.join(os.getcwd(),"Output_Artifacts")
make_directory([OUTPUT_PATH])

'\nDescription: Creating a directory to save the defense artifacts\n'

In [75]:
"""
Description: Download the Vulnerability Report
"""
if my_status == "success":
    output_conf = ais.OutputConf(report_type=ais.get_type("report", "vulnerability"),
                                 file_format=ais.get_type("file_format", "pdf"),
                                 save_folder_path=OUTPUT_PATH)
        
    my_report = client.save_job_report(job_id=my_job_id, output_config=output_conf)

'\nDescription: Download the Vulnerability Report\n'

In [77]:
"""
Description: Download the Defense Reports
"""
if my_status == "success":
    output_conf = ais.OutputConf(report_type=ais.get_type("report", "defense"),
                                 file_format=ais.get_type("file_format", "pdf"),
                                 save_folder_path=OUTPUT_PATH)
        
    my_report = client.save_job_report(job_id=my_job_id, output_config=output_conf)

'\nDescription: Download the Defense Reports\n'

In [80]:
"""
Description: Download the Defense artifacts: Model
"""
if my_status == "success":
    output_conf = ais.OutputConf(report_type=ais.get_type("report", "defense_artifact"),
                                 file_format=ais.get_type("file_format", "pdf"),
                                 save_folder_path=OUTPUT_PATH)
        
    my_report = client.save_job_report(job_id=my_job_id, output_config=output_conf)

'\nDescription: Download the Defense artifacts: Model\n'

In [82]:
"""
Description: Download the Attack Samples
"""
if my_status == "success":
    output_conf = ais.OutputConf(report_type=ais.get_type("report", "attack_samples"),
                                 save_folder_path=OUTPUT_PATH)
        
    my_report = client.save_job_report(job_id=my_job_id, output_config=output_conf)

'\nDescription: Download the Attack Samples\n'

In [2]:
def zip_extractor(file, extract_path=None, delete_zip=False):
    """
    extract zip file to the given path

    Parameters
    ----------
    file : path of zip file
    extract_path : path to extract zip file, default considered parent directory
    delete_zip: True, delete zip file after unzipping it

    Returns
    -------
    None.
    """
    if extract_path is None:
        extract_path = os.path.dirname(file)
    print("Extracting : {}".format(file))
    zf = zipfile.ZipFile(file=file, mode='r')
    zf.extractall(extract_path)
    zf.close()
    if delete_zip:
        os.remove(file)
        print("{} removed successfully.".format(file))


In [9]:
"""
Description: Extracting defense artifact
"""
zip_extractor(file=os.path.join(report_path, 'Defense_artifact.zip'))

'\nDescription: Extracting defense artifact\n'

In [11]:
"""
Description: Load defense model
"""
defense_model_path = os.path.join(report_path, 'defense_model.pkl')
defense_model = pickle.load(open(defense_model_path,'rb'))

'\nDescription: Load defense model\n'

In [10]:
"""
Description: Extracting attack sample
"""
zip_extractor(file=os.path.join(report_path, 'Attack_samples.zip'))

'\nDescription: Extracting attack sample\n'

In [5]:
"""
Description: attack_data visualization
"""

attack_data = pd.read_csv(os.path.join(report_path,"attack_samples.csv"))

In [6]:
attack_data.head()

,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
0,0.093354,0.730612,0.653131,0.791850,0.0,0.990764,0.996411,0.992500,0.121563,0.000000,0.991511,0.000000,0.502935,0.480186,0.776534,0.550559,0.035653,0.000000
1,0.121005,0.000000,0.693062,0.801084,0.0,1.000000,0.000000,0.000000,0.317266,0.002381,1.000000,0.000000,0.511468,0.663929,0.358310,0.340315,0.776495,0.843138
2,0.525601,0.009095,0.309480,0.199172,0.0,0.982245,0.003929,0.982974,0.274631,0.000000,0.973405,0.019302,0.514480,0.912410,0.618720,0.524683,0.990246,0.809770
3,0.031091,0.016219,0.627101,0.563320,0.0,0.003398,0.029189,0.009059,0.259990,0.000000,0.972987,0.000000,0.502052,0.956374,0.623677,0.320410,0.963832,0.956310
4,0.458831,0.178124,0.000000,0.776932,0.0,0.017754,0.025488,0.014317,0.256228,0.000000,0.990683,0.000000,0.498007,0.680622,0.364213,0.343457,0.787258,0.827779
